In [1]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#hyper-parameter
input_size = 784
num_epoch = 5
num_classes = 10
batch_size = 64
learning_rate = 0.001

device = ('cuda' if torch.cuda.is_available() else 'cpu')

#download mnist dataset
train_dataset = torchvision.datasets.MNIST(root='E:\\code\\pytorch\\data\\MNIST',
                                          train=True, transform=torchvision.transforms.ToTensor(),download=True)

test_dataset = torchvision.datasets.MNIST(root='E:\\code\\pytorch\\data\\MIST',
                                         train=False,transform=torchvision.transforms.ToTensor())

#Dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,shuffle=False,
                                           batch_size=batch_size, num_workers=0)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, shuffle=False,
                                         batch_size=batch_size, num_workers=0)

        
model = torch.nn.Sequential(torch.nn.Linear(input_size,num_classes),
                           torch.nn.Softmax())

#loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(),lr=learning_rate)

#train and test model
total_step = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        #reshape iamge to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        #forwad pass
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        #backward and optimizer
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if i % 500 == 0:
            print('Epoch {}/{},step {}/{}, loss: {:.4f}'
                 .format(epoch,num_epoch,i,total_step,loss.item()))
#test model
#in this phase ,we do not need to compute gradients
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, pred = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (pred == labels).sum()
        
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct/total))

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0/5,step 0/938, loss: 2.2974
Epoch 0/5,step 500/938, loss: 1.6549
Epoch 1/5,step 0/938, loss: 1.5832
Epoch 1/5,step 500/938, loss: 1.6140
Epoch 2/5,step 0/938, loss: 1.5527
Epoch 2/5,step 500/938, loss: 1.6025
Epoch 3/5,step 0/938, loss: 1.5394
Epoch 3/5,step 500/938, loss: 1.5957
Epoch 4/5,step 0/938, loss: 1.5316
Epoch 4/5,step 500/938, loss: 1.5909
Accuracy of the model on the 10000 test images: 92 %


In [5]:
torch.save(model.state_dict(),'softmax_model.ckpt')

在pytorch-tutorial里，其中的LR模型直接使用torch.nn.Linear进行训练，个人没看懂，觉得有点奇怪，并没有通过sigmoid或者softmax进行非线性激活，索引自己的改成了softmax的激活函数，进行分类。

还有一个需要注意的是：在pytorch中,torch.nn和torch.nn.functional的区别：

torch.nn中是类，而torch.nn.functional是函数，torch.nn中的类是使用函数包装好的

换句话说，nn.functional中的函数仅仅定义了一些具体的基本操作，不能构成PyTorch中的一个layer。当你需要自定义一些非标准layer时，可以在其中调用nn.functional中的操作。

同时保留这两个的原因是因为，构建网络层的时候使用 类进行构建，看起来足够简洁明了，同时避免了很多的参数定义，但是在做一些简单的操作时，也需要直接构建一个类，太过麻烦，这是就可以使用函数进行简单的操作，而且，当你需要自定义一些网络的层的时候，可以使用函数进行定义。